In [9]:
import pandas as pd

<h2> <b> BASE de ENTRADAS </b> <h2>

### Leitura e Compreensão de colunas

In [ ]:
df_entrada = pd.read_csv("C:\\Users\\jvmm1\\OneDrive\\Documentos\\PJ\\base_entradas_202407142340.csv")
print(df_entrada.columns)

In [ ]:
df_entrada[['ind_oper', 'cod_pais', 'cod_mod', 'cod_sit', 'num_doc', 'num_item','ind_mov','vl_bc_icms_st', 'vl_icms_st', 'aliq_efetiva_icms_st','vl_ipi', 'aliq_efetiva_ipi', 'cfop_cst', 'row_index', 'year','day', 'hour', 'minute','rown']].describe()

In [ ]:
df_entrada = df_entrada[[col for col in df_entrada if df_entrada[col].nunique() > 1]] # retirando colunas para o qual os valores são apenas 1, para compreensão da variação dos dados
df_entrada

In [ ]:
df_entrada[["id_cnpj","cnpj","fornecedor"]].head()
df_entrada["id_dt_fin"] = pd.to_datetime(df_entrada["id_dt_fin"])
df_entrada["id_dt_fin"]

Observando as relações de cnpjs, verifica-se o segundo como mais relevante (linkado ao fornecedor). Agora transformemos alguns dados de id e intervalos de tempo para facilitar visualização.

In [ ]:
df_entrada["month"] = df_entrada["id_dt_fin"].dt.month
df_entrada.drop(["id_dt_ini","id_dt_fin"],axis=1,inplace=True)
df_entrada.head()

In [ ]:
import re
df_cnpj = pd.read_csv("cnpjs.csv")
df_entrada["cnpj"].isna().sum()
df_cnpj["fornecedor"] = df_cnpj["branch_name"]
df_entrada = pd.merge(df_entrada, df_cnpj[['fornecedor', 'cnpj']], left_on='fornecedor', right_on='fornecedor', how='left')
df_entrada["cnpj"] = df_entrada['cnpj_x'].fillna(df_entrada['cnpj_y'])

In [ ]:
df_entrada.drop(["cnpj_x","cnpj_y"],axis=1,inplace=True)
df_entrada.head()

In [ ]:
def format_date(date_str):
    day = date_str[:-6]
    month = date_str[-6:-4]
    year = date_str[-4:]
    return f"{day}-{month}-{year}"

# Apply the function to the column
df_entrada['dt_e_s'] = pd.to_datetime(df_entrada['dt_e_s'].astype(str).apply(format_date),dayfirst=True)
df_entrada['dt_e_s']

Também transformando o cnpj em algo padronizado de acordo com o formato padrão.

In [ ]:
df_entrada["cnpj"] = df_entrada["cnpj"].astype("str")
def format_cnpj(cnpj_str):
    # Remove all non-numeric characters
    digits = ''.join(filter(str.isdigit, cnpj_str))
    # Format the string into the CNPJ pattern
    formatted = f"{digits[:2]}.{digits[2:5]}.{digits[5:8]}/{digits[8:12]}-{digits[12:14]}"
    return formatted

df_entrada["cnpj"] = df_entrada["cnpj"].apply(format_cnpj)
df_entrada.head()

In [ ]:
df_entrada = df_entrada.drop("nome_uf_dest",axis=1)
df_entrada.columns # verificar colunas que permaneceram

Agora, vale fazer análise de o que cada coluna significaria para reunião dos dados que viriam a ser mais importantes em geral.

In [ ]:
df_entrada[["cod_sit","cod_sit_doc"]].value_counts()
# cod_sit_doc mais importante por apresentar descrição
df_entrada["ser"] # ser indica série do doc fiscal
df_entrada["num_doc"] # indica o número do documento fiscal
df_entrada["chv_nfe"] # indica chave da nota fiscal eletrônica
df_entrada["chave_check"] #--> criptografia
df_entrada["dt_doc"] = pd.to_datetime(df_entrada["dt_doc"])
df_entrada[["month","dt_doc"]].value_counts()

In [ ]:
df_entrada[df_entrada["dt_doc"].dt.month != df_entrada["month"]][["dt_doc","month"]]
# os dts de emissão de documento podem ser anteriores ao fim, mas nunca posteriores

In [ ]:
(df_entrada["dt_e_s"].dt.month == df_entrada["month"]).value_counts() # datas de entrada ou saída sempre no mês respectivo
df_entrada["reg_origem_entrada"].value_counts() # registro da origem da entrada --> tipos C170 e C190 ligados a C100
# vários códigos e valores de impostos
df_entrada["num_item"] # número do item sequenciado
df_entrada["cod_item"] # código item
df_entrada[["descr_compl","cod_item"]].value_counts() # código e descrição do item estão totalmente ligados
df_entrada["ncm"].value_counts() # Mercosul
df_entrada["sub_classificacao_gerencial"].value_counts() # descrições mais detalhadas
df_entrada["classificacao_gerencial"].value_counts() # classificação dos itens comercializados pelas notas fiscais

In [ ]:
df_entrada[["qtd","unid","vl_item","vl_desc","classificacao_gerencial"]] # unid como código de unidade de medida

In [ ]:
df_entrada["cst_icms"].value_counts() #Indica código da nota fiscal com código de icms
df_entrada[["cfop","cfop_cst"]].value_counts() # Código Fiscal de Operação e Prestação
df_entrada['tipo'].value_counts() # tipo da localização
df_entrada['origem'].value_counts() # tipo de transação em relação à localização
df_entrada['cod_nat'].value_counts() # natureza da operação

In [ ]:
df_entrada[['analise_credito', 'row_index', 'ncm_abrev',
       'cod_ncm_desc', 'cod_cst_desc']]

In [ ]:
df_entrada.columns

Refletindo sobre as colunas de classificação ou objetos que podem ser mais importantes de acordo com a teoria de tributação e impostos, verificamos aqueles relacionados aos elementos centrais que regem as notas fiscais:
<li> <b> Emitente:</b> Informações de fornecedor ("fornecedor"), CNPJ ("cnpj") e características do emitente ("ind_emit");
<li> <b> Comprador:</b> Informações de cnpj do comprador ("id_cnpj");
<li> <b> Descrição:</b> Diversos códigos dizem respeito ao tipo de comercialização, item e demais classificações, mas destaca-se a classificação ("classificacao_gerencial" e "cod_ncm_desc"), descrição ("descr_compl") e códigos de tipo de aquisição e entrada("reg_origem_entrada");
<li> <b> Dados de localização e tempo:</b> Para cálculos, também é relevante o estado ("uf") ou o tipo de entrada (se é nacional ou internacional, "cst_desc"), além de dados de mês, dia e ano; <br>
Além disso, diversos códigos, como "cfop" e "ncm", relativos aos códigos gerais e de Mercosul para fins legais, e "chv_nfe" para identificar nota fiscal eletrônica.

### Análise Univariada de Variáveis numéricas

Agora, sabendo o significado das colunas e quais podem ser mais bem utilizadas, podemos partir pras variáveis numéricas e ver como elas se distribuem. Algumas apresentam valores únicos e nulos, o que fez essas colunas serem desconsideradas para verificar distribuição, mas podem ser relevantes para pesquisas e construção do modelo.

As colunas que foram desconsideradas seriam 'ind_oper', 'cod_pais', 'cod_mod', 'cod_sit', 'num_doc', 'num_item','ind_mov','vl_bc_icms_st', 'vl_icms_st', 'aliq_efetiva_icms_st','vl_ipi', 'aliq_efetiva_ipi', 'cfop_cst', 'row_index', 'year','day', 'hour', 'minute' e 'rown'. Algumas são inúteis em contexto geral, por serem ids totais que não refletem informações relevantes, enquantos outras podem ser relevantes em respostas como ano, valor de códigos específicos a serem pesquisados ou  valores de alíquotas (que muitas vezes são fixos). Os valores de substituição tributária portanto, sendo nulos, foram desconsiderados.

In [ ]:
df_entrada["month_doc"] = df_entrada["dt_doc"].dt.month
df_entrada.drop("dt_doc",axis=1,inplace=True)
df_entrada["month_e_s"] = df_entrada["dt_e_s"].dt.month
df_entrada.drop("dt_e_s",axis=1,inplace=True)
df_entrada.describe().columns

In [ ]:
df_entrada.describe()

In [ ]:
import plotly.express as px

# Specifying the columns of interest
columns = ['qtd', 'vl_item', 'vl_desc', 'cst_icms', 'vl_bc_icms', 'vl_icms', 'aliq_efetiva_icms',
       'vl_bc_ipi','month', 'month_doc',
       'month_e_s']

# Creating individual boxplots for each column
for column in columns:
    fig = px.box(df_entrada, y=column, title=f'Boxplot for {column}')
    fig.show()


A partir dos boxplots, podemos tirar algumas conclusões sobre as distribuições de variáveis numéricas que são contínuas. Apenas "num_item" significa uma identificação, uma vez que foram desconsiderados os ids para consideração de distribuição e compreensão dos valores existentes. Seguem observações:
<li> <b> Meses: </b> as datas de início, fim e entrada seguem distribuição bastante equalitária, concentrando de maneira pequena nos meses iniciais do ano. 
<li> <b> Valores: </b> Dos mais de 470 mil registros e notas fiscais registradas, observa-se uma grande distribuição que vai de 0 a 120 mil reais, ainda que a grande maioria esteja concentrada em menores valores. Isso se repete nas bases de cálculo de icms (único imposto presente já que são entradas de comércio) e no próprio valor do icms, que varia de acordo com a alíquota efetiva apresentada (que também varia devido aos estados onde estão presentes/de origem).
<li> Ainda sobre valores, os descontos são muitas vezes nulos, mas existem. As quantidades de itens de cada classificação gerencial também são variáveis e podem ser utilizados para cálculo.
<br><br>
Logo todos esses dados são relevantes para a análise.

<h2> <b> BASE de APURAÇÃO de AJUSTE </b> <h2>

In [ ]:
df_ajuste = pd.read_csv("apuracao_ajuste_202407142338.csv")
df_ajuste.drop("rown",axis=1,inplace=True)
df_ajuste

In [ ]:
display(df_ajuste["uf"].value_counts())
df_ajuste["uf"].isna().sum()

In [ ]:
df_ajuste["origem_ajuste"].value_counts()

<h2> <b> BASE de APURAÇÃO de ICMS </b> <h2>

In [ ]:
df_icms = pd.read_csv("apuracao_icms_202407142340.csv")
df_icms

In [ ]:
df_icms.describe()

<h2> <b> BASE de APURAÇÃO de ICMS ST</b> <h2>

In [ ]:
df_icms_st = pd.read_csv("apuracao_icms_st_202407142340.csv")
df_icms_st

In [ ]:
df_icms_st.columns

In [ ]:
df_icms_st.describe()

<h2> <b> BASE de CNPJ</b> <h2>

In [ ]:
df_cnpj = pd.read_csv("cnpjs.csv")
df_cnpj

In [ ]:
df_cnpj.columns

In [ ]:
df_cnpj.describe()

In [ ]:
df_nfs_faltantes = pd.read_csv("nfs_faltantes_consolidado_202407142339.csv")
df_nfs_faltantes

In [ ]:
df_nfs_faltantes.drop_duplicates(["filial","competencia","cnpj"])

In [32]:
import json
import pandas as pd

# Load the JSON file containing the column mappings
with open(r"C:\Users\jvmm1\OneDrive\Documentos\PJ\NutaxProj\Dados Taxboard - Paula Torres\jsons\dicionario_da_base.json", encoding="utf-8") as f:
    data = json.load(f)

# Assuming df_entrada is already defined, rename its columns using the loaded mappings
df_entrada = df_entrada.rename(columns=data)

# Now df_entrada will have its columns renamed according to the mappings in 'data'

In [ ]:
df_entrada.describe().columns